# Audiobooks Business Usecase

## Problem

We are given data from an Audiobook app. Logically, it relates only to the audio versions of books. Each customer in the database has made a purchase at least once, that's why he/she is in the database. We want to create a machine learning algorithm based on our available data that can predict if a customer will buy again from the Audiobook company.

The main idea is that if a customer has a low probability of coming back, there is no reason to spend any money on advertizing to him/her. If we can focus our efforts ONLY on customers that are likely to convert again, we can make great savings. Moreover, this model can identify the most important metrics for a customer to come back again. Identifying new customers creates value and growth opportunities.

### Dataset Description
We have a .csv summarizing the data. 
There are several variables: 
- Customer ID, 
- Book length in mins_avg (average of all purchases), 
- Book length in minutes_sum (sum of all purchases), 
- Price Paid_avg (average of all purchases), 
- Price paid_sum (sum of all purchases), 
- Review (a Boolean variable), 
- Review (out of 10), 
- Total minutes listened, 
- Completion (from 0 to 1), 
- Support requests (number),  
- Last visited minus purchase date (in days).

So these are the inputs (excluding **"Customer ID"**, as it is completely arbitrary. It's more like a name, than a number).

The targets are a Boolean variable (so 0, or 1). We are taking a period of 2 years in our inputs, and the next 6 months as targets. So, in fact, we are predicting if: based on the last 2 years of activity and engagement, a customer will convert in the next 6 months. 6 months sounds like a reasonable time. If they don't convert after 6 months, chances are they've gone to a competitor or didn't like the Audiobook way of digesting information. 

## Task
Create a machine learning algorithm, which is able to predict if a customer will buy again. 

This is a classification problem with two classes: won't buy and will buy, represented by 0s and 1s. 

Now let's get started..

### Import the relevant libraries

In [1]:
import numpy as np
import tensorflow as tf
# We will use the StandardScaler module, so we can later deploy the model
from sklearn.preprocessing import StandardScaler

import pickle

### Extract the data from the csv

In [2]:
# Load the data
raw_csv_data = np.loadtxt('data/Audiobooks_data.csv',delimiter=',')

In [3]:
# The inputs are all columns in the csv, except for the first one and the last one
# The first column is the arbitrary ID, while the last contains the targets

unscaled_inputs_all = raw_csv_data[:,1:-1]

# The targets are in the last column.
targets_all = raw_csv_data[:,-1]

### Balance the dataset

In [4]:
# Since we are dealing with real life data, we will need to preprocess it a bit.

# Count how many targets are 1 (meaning that the customer did convert)
num_one_targets = int(np.sum(targets_all))

# Set a counter for targets that are 0 (meaning that the customer did not convert)
zero_targets_counter = 0

# We want to create a "balanced" dataset, so we will have to remove some input/target pairs.
# Declare a variable that will do that:
indices_to_remove = []

# Count the number of targets that are 0. 
# Once there are as many 0s as 1s, mark entries where the target is 0.
for i in range(targets_all.shape[0]):
    if targets_all[i] == 0:
        zero_targets_counter += 1
        if zero_targets_counter > num_one_targets:
            indices_to_remove.append(i)

# Create two new variables, one that will contain the inputs, and one that will contain the targets.
# We delete all indices that we marked "to remove" in the loop above.
unscaled_inputs_equal_priors = np.delete(unscaled_inputs_all, indices_to_remove, axis=0)
targets_equal_priors = np.delete(targets_all, indices_to_remove, axis=0)

### Standardize the inputs

In [5]:
# Crete a standar scaler object
scaler_deep_learning = StandardScaler()
# Fit and transform the original data
# Essentially, we calculate and STORE the mean and variance of the data in the scaler object
# At the same time we standardize the data using this information
# Note that the mean and variance remain recorded in the scaler object
scaled_inputs = scaler_deep_learning.fit_transform(unscaled_inputs_equal_priors)

### Shuffle the data

In [6]:
# When the data was collected it was actually arranged by date
# Shuffle the indices of the data, so the data is not arranged in any way when we feed it.
# Since we will be batching, we want the data to be as randomly spread out as possible
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

# Use the shuffled indices to shuffle the inputs and targets.
shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = targets_equal_priors[shuffled_indices]

### Split the dataset into train, validation, and test

In [7]:
# Count the total number of samples
samples_count = shuffled_inputs.shape[0]

# Count the samples in each subset, assuming we want 80-10-10 distribution of training, validation, and test.
# Naturally, the numbers are integers.
train_samples_count = int(0.8 * samples_count)
validation_samples_count = int(0.1 * samples_count)

# The 'test' dataset contains all remaining data.
test_samples_count = samples_count - train_samples_count - validation_samples_count

# Create variables that record the inputs and targets for training
# In our shuffled dataset, they are the first "train_samples_count" observations
train_inputs = shuffled_inputs[:train_samples_count]
train_targets = shuffled_targets[:train_samples_count]

# Create variables that record the inputs and targets for validation.
# They are the next "validation_samples_count" observations, folllowing the "train_samples_count" we already assigned
validation_inputs = shuffled_inputs[train_samples_count:train_samples_count+validation_samples_count]
validation_targets = shuffled_targets[train_samples_count:train_samples_count+validation_samples_count]

# Create variables that record the inputs and targets for test.
# They are everything that is remaining.
test_inputs = shuffled_inputs[train_samples_count+validation_samples_count:]
test_targets = shuffled_targets[train_samples_count+validation_samples_count:]

# We balanced our dataset to be 50-50 (for targets 0 and 1), but the training, validation, and test were 
# taken from a shuffled dataset. Check if they are balanced, too. 
# Note that each time when we rerun this code, we will get different values, as each time they are shuffled randomly.

# Print the number of targets that are 1s, the total number of samples, and the proportion for training, validation, and test.
print(np.sum(train_targets), train_samples_count, np.sum(train_targets) / train_samples_count)
print(np.sum(validation_targets), validation_samples_count, np.sum(validation_targets) / validation_samples_count)
print(np.sum(test_targets), test_samples_count, np.sum(test_targets) / test_samples_count)

1797.0 3579 0.5020955574182733
210.0 447 0.4697986577181208
230.0 448 0.5133928571428571


### Save the three datasets in *.npz

In [8]:
# Save the three datasets in *.npz.
np.savez('Audiobooks_data_train', inputs=train_inputs, targets=train_targets)
np.savez('Audiobooks_data_validation', inputs=validation_inputs, targets=validation_targets)
np.savez('Audiobooks_data_test', inputs=test_inputs, targets=test_targets)

### Save the scaler

In [9]:
# Saving the scalar object, so we can apply in on new data
pickle.dump(scaler_deep_learning, open('scaler_deep_learning.pickle', 'wb'))

## Create the machine learning algorithm



### Data

In [10]:
# let's create a temporary variable npz, where we will store each of the three Audiobooks datasets
npz = np.load('Audiobooks_data_train.npz')

# we extract the inputs using the keyword under which we saved them
# to ensure that they are all floats, let's also take care of that
train_inputs = npz['inputs'].astype(np.float)
# targets must be int because of sparse_categorical_crossentropy (we want to be able to smoothly one-hot encode them)
train_targets = npz['targets'].astype(np.int)

# we load the validation data in the temporary variable
npz = np.load('Audiobooks_data_validation.npz')
# we can load the inputs and the targets in the same line
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# we load the test data in the temporary variable
npz = np.load('Audiobooks_data_test.npz')
# we create 2 variables that will contain the test inputs and the test targets
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

### Model
Outline, optimizers, loss, early stopping and training

In [11]:
# Optionally set the input size. We won't be using it, but in some cases it is beneficial
# input_size = 10
# Set the output size
output_size = 2
# Use same hidden layer size for both hidden layers. Not a necessity.
hidden_layer_size = 50
    
# define how the model will look like
model = tf.keras.Sequential([
    # tf.keras.layers.Dense is basically implementing: output = activation(dot(input, weight) + bias)
    # it takes several arguments, but the most important ones for us are the hidden_layer_size and the activation function
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 2nd hidden layer
    # the final layer is no different, we just make sure to activate it with softmax 
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])


### Choose the optimizer and the loss function

# we define the optimizer we'd like to use, 
# the loss function, 
# and the metrics we are interested in obtaining at each iteration
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

### Training
# That's where we train the model we have built.

# set the batch size
batch_size = 100

# set a maximum number of training epochs
max_epochs = 100

# set an early stopping mechanism
# let's set patience=2, to be a bit tolerant against random validation loss increases
early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

# fit the model
# note that this time the train, validation and test data are not iterable
model.fit(train_inputs, # train inputs
          train_targets, # train targets
          batch_size=batch_size, # batch size
          epochs=max_epochs, # epochs that we will train for (assuming early stopping doesn't kick in)
          # callbacks are functions called by a task when a task is completed
          # task here is to check if val_loss is increasing
          callbacks=[early_stopping], # early stopping
          validation_data=(validation_inputs, validation_targets), # validation data
          verbose = 2 # making sure we get enough information about the training process
          )  

Train on 3579 samples, validate on 447 samples
Epoch 1/100
3579/3579 - 0s - loss: 0.5762 - accuracy: 0.7608 - val_loss: 0.4560 - val_accuracy: 0.8635
Epoch 2/100
3579/3579 - 0s - loss: 0.3987 - accuracy: 0.8718 - val_loss: 0.3455 - val_accuracy: 0.8881
Epoch 3/100
3579/3579 - 0s - loss: 0.3339 - accuracy: 0.8854 - val_loss: 0.3060 - val_accuracy: 0.8881
Epoch 4/100
3579/3579 - 0s - loss: 0.3064 - accuracy: 0.8905 - val_loss: 0.2872 - val_accuracy: 0.8926
Epoch 5/100
3579/3579 - 0s - loss: 0.2893 - accuracy: 0.8961 - val_loss: 0.2721 - val_accuracy: 0.8949
Epoch 6/100
3579/3579 - 0s - loss: 0.2766 - accuracy: 0.8986 - val_loss: 0.2617 - val_accuracy: 0.8926
Epoch 7/100
3579/3579 - 0s - loss: 0.2701 - accuracy: 0.9022 - val_loss: 0.2553 - val_accuracy: 0.9038
Epoch 8/100
3579/3579 - 0s - loss: 0.2633 - accuracy: 0.9030 - val_loss: 0.2523 - val_accuracy: 0.9060
Epoch 9/100
3579/3579 - 0s - loss: 0.2616 - accuracy: 0.9028 - val_loss: 0.2525 - val_accuracy: 0.9083
Epoch 10/100
3579/3579 - 0

## Test the model

After training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

It is very important to realize that fiddling with the hyperparameters overfits the validation dataset. 

The test is the absolute final instance. We should not test before we're completely done with adjusting your model.

If we adjust our model after testing, we will start overfitting the test dataset, which will defeat its purpose.

In [12]:
# declare two variables that are going to contain the two outputs from the evaluate function
# they are the loss (which is there by default) and whatever was specified in the 'metrics' argument when fitting the model
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

448/448 [==============================] - 0s 156us/sample - loss: 0.2610 - accuracy: 0.9018


In [13]:
# Print the result in neatly formatted
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.26. Test accuracy: 90.18%


## Obtain the probability for a customer to convert

In [14]:
# We can predict the probability of each class using the 'predict' method
# The output comes in a scientific format
model.predict(test_inputs).round(2)[:15] # Checking the first 15 predictions

array([[0.28, 0.72],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.35, 0.65],
       [0.  , 1.  ],
       [0.  , 1.  ],
       [0.91, 0.09],
       [0.12, 0.88],
       [1.  , 0.  ],
       [0.  , 1.  ],
       [0.96, 0.04],
       [0.39, 0.61],
       [0.16, 0.84],
       [0.1 , 0.9 ]], dtype=float32)

In [15]:
# In this case we would like to know if the customer will convert again and thus referring to second column
# Once more, we can round to 0 digits, to achieve only 0s or 1s
model.predict(test_inputs)[:,1].round(0)

array([1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 1., 1.,
       1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 0., 1., 1., 1., 0.,
       1., 1., 1., 0., 1., 1., 1., 0., 1., 1., 0., 1., 0., 0., 1., 1., 0.,
       1., 0., 0., 1., 0., 0., 1., 1., 1., 1., 0., 1., 0., 0., 0., 1., 0.,
       1., 1., 0., 0., 0., 0., 1., 1., 0., 1., 1., 1., 1., 0., 0., 0., 1.,
       0., 1., 0., 0., 0., 0., 1., 1., 1., 1., 0., 1., 1., 1., 0., 1., 1.,
       0., 0., 1., 0., 1., 1., 1., 1., 0., 1., 1., 0., 1., 1., 1., 1., 0.,
       0., 1., 1., 1., 1., 0., 0., 1., 1., 0., 0., 1., 0., 1., 1., 0., 0.,
       1., 0., 1., 1., 0., 0., 0., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0.,
       1., 1., 1., 0., 0., 0., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
       1., 1., 0., 0., 1., 1., 1., 0., 1., 0., 1., 1., 1., 1., 0., 1., 0.,
       0., 0., 1., 0., 0., 0., 1., 1., 1., 0., 0., 1., 0., 1., 1., 1., 0.,
       0., 0., 0., 1., 1., 0., 1., 1., 1., 0., 0., 0., 1., 0., 0., 1., 1.,
       0., 0., 0., 1., 1.

In [16]:
np.argmax(model.predict(test_inputs),axis=1)
# A much better approach here would be to use argmax (arguments of the maxima)
# Argmax indicates the position of the highest argument row-wise or column-wise
# In our case, we want to know which COLUMN has the higher argument (probability), therefore we set axis=1 (for columns)
# The output would be the column ID with the highest argument for each observation (row)
# For instance, the first observation (in our output) was [0.28,0.72]
# np.argmax([0.28,0.72], axis=1) would find that 0.72 is the higher argument (higher probability) and return 1
# This method is great for multi-class problems as it is independent of the number of classes

array([1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1,
       0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1,
       0, 1, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1,

## Save the model

In [17]:
# Finally we save the model using the built-in method TensorFlow method
# We choose the name and the file extension
# Since the HDF format is optimal for large numerical objects, that's our preferred choice here (and the one recommended by TF)
# The proper extension is .h5 to indicate HDF, version 5
model.save('audiobooks_model.h5') 

# Predicting on new data

## Load the scaler and the model

In [18]:
# To load the scaler we use the pickle method load
scaler_deep_learning = pickle.load(open('scaler_deep_learning.pickle', 'rb'))
# To load the model, we use the TensorFlow (Keras) function relevant for the operation
model = tf.keras.models.load_model('audiobooks_model.h5')

# Note that since we did not specify the input shape of our inputs in the modeling part, we get a warning
# For feed-forward neural networks such as ours that is not an issue

## Load the new data

In [19]:
# The new data is located in 'data/New_Audiobooks_Data.csv'
# To keep everything as before, we must specify the delimiter explicitly
raw_data = np.loadtxt('data/New_Audiobooks_Data.csv',delimiter=',')
# We are interested in all data except for the first column (ID)
# Note that there are no targets in this CSV file (we don't know the behavior of these clients, yet!)
new_data_inputs = raw_data[:,1:]

## Predict the probability of a customer to convert

In [20]:
# Scale the new data in the same way we scaled the train data
new_data_inputs_scaled = scaler_deep_learning.transform(new_data_inputs)

In [21]:
# Predict the probability of each customer to convert
# Here we have also taken only the second column and rounded to 2 digits after the dot
model.predict(new_data_inputs_scaled)[:,1].round(2)

array([0.02, 0.  , 0.07, 1.  , 0.01, 0.05, 0.04, 0.08, 0.03, 0.73, 0.  ,
       0.75, 0.89, 0.  , 0.09, 0.09, 0.83, 0.68, 0.77, 0.95, 1.  , 1.  ,
       1.  , 0.26, 0.  , 1.  , 0.48, 0.  , 1.  , 1.  ], dtype=float32)

In [22]:
# Using argmax which is independent of the number of classes
np.argmax(model.predict(new_data_inputs_scaled),1)

array([0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1,
       1, 0, 0, 1, 0, 0, 1, 1], dtype=int64)